In [1]:
# spark
import findspark
findspark.init()
import pyspark
pyspark.__version__

'3.1.2'

In [2]:
import pandas as pd
import PIL
from PIL import Image
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

In [3]:
# Versions
print('Libraries used:')
print('Python        : ' + sys.version)
print('pyspark       : ' + pyspark.__version__)
print('PIL           : ' + PIL.__version__)
print('Numpy         : ' + np.__version__)
print('Pandas        : ' + pd.__version__)

Libraries used:
Python        : 3.8.8 (tags/v3.8.8:024d805, Feb 19 2021, 13:18:16) [MSC v.1928 64 bit (AMD64)]
pyspark       : 3.1.2
PIL           : 9.0.1
Numpy         : 1.22.3
Pandas        : 1.4.1


In [4]:
PATH = os.getcwd()
PATH_Data = PATH+'/Sample'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        c:\Users\steph\Documents\Formation_Data_Scientist\P8_Lanchec_Stephane
PATH_Data:   c:\Users\steph\Documents\Formation_Data_Scientist\P8_Lanchec_Stephane/Sample
PATH_Result: c:\Users\steph\Documents\Formation_Data_Scientist\P8_Lanchec_Stephane/Results


In [5]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

In [6]:
sc = spark.sparkContext

In [7]:
spark

In [8]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [9]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------------------------------------------------------------+---------------+
|path                                                                                                         |label          |
+-------------------------------------------------------------------------------------------------------------+---------------+
|file:/c:/Users/steph/Documents/Formation_Data_Scientist/P8_Lanchec_Stephane/Sample/cabbage_white_1/r0_172.jpg|cabbage_white_1|
|file:/c:/Users/steph/Documents/Formation_Data_Scientist/P8_Lanchec_Stephane/Sample/cabbage_white_1/r0_170.jpg|cabbage_white_1|
|file:/c:/Users/steph/Documents/Formation_Data_Scientist/P8_Lanchec_Stephane/Sample/cabbage_white_1/r0_168.jpg|cabbage_white_1|
|file:/c:/Users/steph/

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [11]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

C:\Spark\spark-3.1.2-bin-hadoop3.2\python\pyspark\sql\pandas\functions.py:389: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [15]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [16]:
print(PATH_Result)

c:\Users\steph\Documents\Formation_Data_Scientist\P8_Lanchec_Stephane/Results


In [17]:
features_df.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\steph\anaconda3\lib\site-packages\numpy\core\__init__.py", line 22, in <module>
    from . import multiarray
  File "C:\Users\steph\anaconda3\lib\site-packages\numpy\core\multiarray.py", line 12, in <module>
    from . import overrides
  File "C:\Users\steph\anaconda3\lib\site-packages\numpy\core\overrides.py", line 7, in <module>
    from numpy.core._multiarray_umath import (
ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 588, in main
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 336, in read_udfs
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\

In [ ]:
sys.executable